### Bayesian Linear Regression using Pyro

The motivation for this example and some of the methods from from the Pyro documentation here: http://pyro.ai/examples/bayesian_regression.html

I will use Bayesian linear regression to analyze the relationship between different concrete attributes and concrete compressive strength.

### Data:
The data come from: "Yeh, I-Cheng (1998). Modeling of strength of high performance concrete using artificial neural networks"

Concrete is the most important material in civil engineering. The concrete compressive strength is a highly dependent of age and ingredients. These ingredients include cement, blast furnace slag, fly ash, water, superplasticizer, coarse
aggregate, and fine aggregate. The actual concrete compressive strength (in MPa) for a given mixture of a specific age
(in days) was determined in a laboratory. Data set consists of n = 1030 measurements of 8 predictors and concrete compressive strength as the response variable

In [1]:
import os
from functools import partial
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import pyro
from pyro.distributions import Normal, Uniform, Delta
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.distributions.util import logsumexp
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive
from pyro.infer.mcmc import MCMC, NUTS
import pyro.optim as optim
import pyro.poutine as poutine

# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)
pyro.set_rng_seed(1)
pyro.enable_validation(True)

In [2]:
df = pd.read_csv("concrete.csv")
df.head()

,cement,blast furnace slag,fly ash,water,superplasticizer,coarse aggregate,fine aggregate,age,compressive strength
0,5.40,0.00,0.0,1.62,0.25,10.40,6.76,28,79.99
1,5.40,0.00,0.0,1.62,0.25,10.55,6.76,28,61.89
2,3.33,1.43,0.0,2.28,0.00,9.32,5.94,270,40.27
3,3.33,1.43,0.0,2.28,0.00,9.32,5.94,365,41.05
4,1.99,1.32,0.0,1.92,0.00,9.78,8.26,360,44.30


In [3]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)
        self.factor = nn.Parameter(torch.tensor(1.))

    def forward(self, x):
        return self.linear(x) + (self.factor * x[:, 0] * x[:, 1]).unsqueeze(1)

p = 8 # number of features
regression_model = RegressionModel(p)

In [4]:
loss_fn = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.Adam(regression_model.parameters(), lr=0.05)
num_iterations = 1000 if not smoke_test else 2
data = torch.tensor(df.values, dtype=torch.float)
x_data, y_data = data[:, :-1], data[:, -1]

def main():
    x_data = data[:, :-1]
    y_data = data[:, -1]
    for j in range(num_iterations):
        # run the model forward on the data
        y_pred = regression_model(x_data).squeeze(-1)
        # calculate the mse loss
        loss = loss_fn(y_pred, y_data)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        if (j + 1) % 50 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))
    # Inspect learned parameters
    print("Learned parameters:")
    for name, param in regression_model.named_parameters():
        print(name, param.data.numpy())

main()

[iteration 0050] loss: 198297.0625
[iteration 0100] loss: 174947.9531
[iteration 0150] loss: 157223.2812
[iteration 0200] loss: 144260.0469
[iteration 0250] loss: 135703.9219
[iteration 0300] loss: 130423.1016
[iteration 0350] loss: 127252.2188
[iteration 0400] loss: 125297.4688
[iteration 0450] loss: 123983.0547
[iteration 0500] loss: 122984.3281
[iteration 0550] loss: 122137.8125
[iteration 0600] loss: 121369.0625
[iteration 0650] loss: 120647.0547
[iteration 0700] loss: 119959.7656
[iteration 0750] loss: 119302.6953
[iteration 0800] loss: 118674.0938
[iteration 0850] loss: 118073.1562
[iteration 0900] loss: 117499.3672
[iteration 0950] loss: 116952.3281
[iteration 1000] loss: 116431.6406
Learned parameters:
factor 2.2542868
linear.weight [[ 8.307957    1.6912518   5.2816205  -5.974391    5.9625874   0.6745347
   0.04105157  0.10338848]]
linear.bias [-0.20299356]


In [5]:
def model(x_data, y_data):
    # weight and bias priors
    w_prior = Normal(torch.zeros(1, 8), torch.ones(1, 8)).to_event(1)
    b_prior = Normal(torch.tensor([[8.]]), torch.tensor([[1000.]])).to_event(1)
    f_prior = Normal(0., 1.)
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior, 'factor': f_prior}
    scale = pyro.sample("sigma", Uniform(0., 10.))
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a nn (which also samples w and b)
    lifted_reg_model = lifted_module()
    with pyro.plate("map", len(x_data)):
        # run the nn forward on data
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, scale),
                    obs=y_data)
        return prediction_mean

In [6]:
from pyro.contrib.autoguide import AutoDiagonalNormal
guide = AutoDiagonalNormal(model)

In [7]:
optim = Adam({"lr": 0.03})
svi = SVI(model, guide, optim, loss=Trace_ELBO(), num_samples=1000)

In [8]:
def train():
    pyro.clear_param_store()
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(x_data, y_data)
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

train()

[iteration 0001] loss: 147.2690
[iteration 0101] loss: 5.2177
[iteration 0201] loss: 6.5852
[iteration 0301] loss: 6.2925
[iteration 0401] loss: 4.5894
[iteration 0501] loss: 6.7542
[iteration 0601] loss: 7.8444
[iteration 0701] loss: 7.6511
[iteration 0801] loss: 6.3876
[iteration 0901] loss: 9.3115


In [9]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

auto_loc tensor([1.5164, 2.1996, 2.3958, 0.3633, 1.2838, 0.4556, 4.2051, 0.8214, 0.8229,
        0.0715, 0.8860], requires_grad=True)
auto_scale tensor([0.4021, 0.3503, 0.4268, 0.3825, 0.5074, 0.5321, 0.6596, 0.2025, 0.2981,
        0.1600, 0.7665], grad_fn=<AddBackward0>)


In [10]:
get_marginal = lambda traces, sites:EmpiricalMarginal(traces, sites)._get_samples_and_weights()[0].detach().cpu().numpy()

def summary(traces, sites):
    marginal = get_marginal(traces, sites)
    site_stats = {}
    for i in range(marginal.shape[1]):
        site_name = sites[i]
        marginal_site = pd.DataFrame(marginal[:, i]).transpose()
        describe = partial(pd.Series.describe, percentiles=[.05, 0.25, 0.5, 0.75, 0.95])
        site_stats[site_name] = marginal_site.apply(describe, axis=1) \
            [["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

def wrapped_model(x_data, y_data):
    pyro.sample("prediction", Delta(model(x_data, y_data)))

posterior = svi.run(x_data, y_data)

In [11]:
trace_pred = TracePredictive(wrapped_model,
                             posterior,
                             num_samples=1000)
post_pred = trace_pred.run(x_data, None)
post_summary = summary(post_pred, sites= ['prediction', 'obs'])

In [12]:
mu = post_summary["prediction"]
y = post_summary["obs"]

In [13]:
predictions = pd.DataFrame({
    "cement": x_data[:, 0],
    "blast_furnace_slag": x_data[:, 1],
    "fly_ash": x_data[:, 2],
    "water": x_data[:, 3],
    "superplasticizer": x_data[:, 4],
    "coarse_aggregate": x_data[:, 5],
    "fine_aggregatte": x_data[:, 6],
    "age": x_data[:, 7],
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
    "y_mean": y["mean"],
    "y_perc_5": y["5%"],
    "y_perc_95": y["95%"],
    "compressive_strength": y_data,
})

We generated predictive samples and look at the posteriors

In [14]:
predictions.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregatte,age,mu_mean,mu_perc_5,mu_perc_95,y_mean,y_perc_5,y_perc_95,compressive_strength
0,5.40,0.00,0.0,1.62,0.25,10.40,6.76,28.0,31.845823,22.778471,41.688886,31.934553,16.109538,48.449003,79.989998
1,5.40,0.00,0.0,1.62,0.25,10.55,6.76,28.0,31.969635,22.891130,41.812694,32.026432,15.681419,47.796537,61.889999
2,3.33,1.43,0.0,2.28,0.00,9.32,5.94,270.0,53.450081,-12.659648,126.761879,53.196125,-15.253030,128.192059,40.270000
3,3.33,1.43,0.0,2.28,0.00,9.32,5.94,365.0,60.455345,-27.642742,159.721344,60.495701,-29.817501,159.699846,41.049999
4,1.99,1.32,0.0,1.92,0.00,9.78,8.26,360.0,54.198627,-32.885677,152.218506,54.323368,-36.315387,152.092482,44.299999
